# Context Manager
- A context manager in Python is responsible for allocating and releasing resources precisely at the desired time. 
- It ensures that resources are set up when needed and then automatically cleaned up or released after use, even if an error occurs. 

### With

In [1]:
file = open("./testfile1.txt", 'w')
try:
    file.write("Context manager test1\nContextlib test1.")
finally:
    file.close()

In [3]:
import os
print(os.listdir())

['.gitignore', '.ipynb_checkpoints', 'context_manager.ipynb', 'copy.ipynb', 'functions.ipynb', 'scope.ipynb', 'testfile1.txt']


In [5]:
with open("./testfile2.txt", 'w') as f:
    f.write("Context manager test1\nContextlib test2.")

In [6]:
print(os.listdir())

['.gitignore', '.ipynb_checkpoints', 'context_manager.ipynb', 'copy.ipynb', 'functions.ipynb', 'scope.ipynb', 'testfile1.txt', 'testfile2.txt']


### `__enter__`, `__exit__`

In [9]:
# Context manager with exception handling

class MyFileWriter():
    def __init__(self, file_name, method):
        print("MyFileWriter started : __init__")
        self.file_obj = open(file_name, method)
        
    def __enter__(self):
        print("MyFileWriter started : __enter__")
        return self.file_obj
    
    def __exit__(self, exc_type, value, trace_back):
        print("MyFileWriter started : __exit__")
        if exc_type:
            print("Logging exxception {}".format((exc_type, value, trace_back)))
        self.file_obj.close()

In [10]:
# with -> init -> enter -> exit
with MyFileWriter('./testfile3.txt', 'w') as f:
    f.write("Context manager test1\nContextlib test3.")

MyFileWriter started : __init__
MyFileWriter started : __enter__
MyFileWriter started : __exit__


In [12]:
# Contextlib: Measure execution

import time

class ExecuteTimer(object):
    def __init__(self, msg):
        self._msg = msg
        
    def __enter__(self):
        self._start = time.monotonic()
        return self._start
    
    # exit from `with`
    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type:
            print("Logging exception {}".format((exc_type, exc_value, exc_traceback)))
        else:
            print("{} : {} s".format(self._msg, time.monotonic() - self._start))
        return True

- v: returned value of the `__enter__`

In [13]:
with ExecuteTimer("Start! job") as v:
    print("Received start monotonic1 : {}".format(v))
    # Execute job
    for i in range(1000000):
        pass

Received start monotonic1 : 1307234.875
Start! job : 0.030999999959021807 s


In [14]:
with ExecuteTimer("Start! job") as v:
    print("Received start monotonic1 : {}".format(v))
    # Execute job
    for i in range(1000000):
        pass
    raise Exception("Raise! Exception!!")

Received start monotonic1 : 1307309.031
Logging exception (<class 'Exception'>, Exception('Raise! Exception!!'), <traceback object at 0x00000110B689C1C0>)


### `__enter__`, `__exit__`, `@contextlib.contextmanager`

In [16]:
import contextlib
import time

# Same with `ExecuteTimer` class
@contextlib.contextmanager
def my_file_writer(file_name, method):
    f = open(file_name, method)
    yield f  # __enter__
    f.close()  # __exit__
    
with my_file_writer("testfile4.txt", 'w') as f:
    f.write("Context manager test4. \nContextlib Test4.")

In [17]:
print(os.listdir())

['.gitignore', '.ipynb_checkpoints', 'context_manager.ipynb', 'copy.ipynb', 'functions.ipynb', 'scope.ipynb', 'testfile1.txt', 'testfile2.txt', 'testfile3.txt', 'testfile4.txt']


In [20]:
# User decorator

@contextlib.contextmanager
def ExecuteTimerDc(msg):
    start = time.monotonic()
    try:  # __enter__
        yield start
    except BaseException as e:
        print("Logging exception: {}: {}".format(msg, e))
        raise
    else:  # __exit__
        print("{} : {}s".format(msg, time.monotonic() - start))
        

with ExecuteTimerDc("Start job!") as v:
    print("Received start monotonic2 : {}".format(v))
    for i in range(100000):
        pass
#     raise ValueError("occured.")

Received start monotonic2 : 1307860.515
Start job! : 0.0s
